In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import csv
import numpy as np

In [3]:
disease_df = pd.read_csv('Diseases_sliced.csv')
compound_df = pd.read_csv('Compounds_sliced.csv')
indication_df = pd.read_csv('CtD_edges_sliced.csv')
print(len(compound_df), len(disease_df))

85 9


In [5]:
df_1 = pd.read_csv('CurGiGaD.csv')
df_2 = pd.read_csv('CdrGiGaD.csv')
df_3 = pd.read_csv('CsCtD.csv')
df_4 = pd.read_csv('CtGaD.csv')
print(len(df_1)+len(df_2)+len(df_3)+len(df_4))

DWPC_df = pd.concat([df_1,df_2,df_3,df_4])

print(len(DWPC_df))

3060
3060


In [6]:
DWPC_df.to_csv("DWPC_New.csv", index = False)
dwpc_df = DWPC_df

In [7]:
##Compound-Disease Pairs

indication_df = indication_df.rename(columns={'doid_id': 'disease_id', 'drugbank_id': 'compound_id'})[['compound_id', 'disease_id', 'category']]
# print(indication_df)

In [8]:
rows = list()
for c, d in itertools.product(compound_df.itertuples(), disease_df.itertuples()):
    row = c.id, c.name, d.doid, d.name
    rows.append(row)
pair_df = pd.DataFrame(rows, columns=['compound_id', 'compound_name', 'disease_id', 'disease_name'])
pair_df = pair_df.merge(indication_df, how='left')
pair_df['status'] = (pair_df.category == 'DM').astype(int)
# pair_df['status'].fillna(0, inplace=True)
# pair_df

In [9]:
with open('1-prepare_compound-disease-pairs.csv', 'wt') as write_file:
    pair_df.to_csv(write_file, sep=',', index=False)

In [10]:
##DWPC
# dwpc_df = pd.read_csv('DWPC.csv')
dwpc_df.head()

,compound_id,drug,disease_id,disease,metapath,treatment,path_count,DWPC
0,DB00244,Mesalazine,DOID:635,acquired immunodeficiency syndrome,CurGiGaD,0,5,0.015643
1,DB00795,Sulfasalazine,DOID:9206,Barrett's esophagus,CurGiGaD,0,2,0.009761
2,DB00550,Propylthiouracil,DOID:9206,Barrett's esophagus,CurGiGaD,0,2,0.009761
3,DB00387,Procyclidine,DOID:9206,Barrett's esophagus,CurGiGaD,0,2,0.009761
4,DB01104,Sertraline,DOID:635,acquired immunodeficiency syndrome,CurGiGaD,0,6,0.009224


In [11]:
dwpc_mat_df = dwpc_df.pivot_table(values = 'path_count', index=['compound_id', 'disease_id'], columns = 'metapath')
dwpc_mat_df.columns = 'PC_' + dwpc_mat_df.columns
dwpc_mat_df = dwpc_mat_df.reset_index()
dwpc_mat_df.head(2)

,compound_id,disease_id,PC_CdrGiGaD,PC_CsCtD,PC_CtGaD,PC_CurGiGaD
0,DB00158,DOID:10652,0,240,1,0
1,DB00158,DOID:11119,0,114,0,0


In [12]:
with open('1-prepare_DWPC.csv', 'wt') as write_file:
    dwpc_mat_df.to_csv(write_file, sep=',', index=False)

In [13]:
df_ctd_edges = pd.read_csv('CtD.csv').head(100)

unique_compounds = list(set(df_ctd_edges['drugbank_id']))
unique_diseases = list(set(df_ctd_edges['doid_id']))

In [15]:
###Degrees
dwpc_df = pd.read_csv('DWPC.csv', encoding='ISO-8859-1')
metaedge_style_df = pd.read_csv('metaedge-styles.csv', encoding='ISO-8859-1')
disease_degree_df = pd.read_excel("degrees_Sliced.xlsx", sheetname='Disease')
compound_degree_df = pd.read_excel("degrees_Sliced.xlsx", sheetname='Compound')

metaedge_to_abbreviation = dict(zip(metaedge_style_df.metaedge, metaedge_style_df.abbreviation))

disease_degree_df = disease_degree_df.rename(columns={'node_id': 'disease_id'}).drop('node_name', axis='columns')
disease_degree_df = disease_degree_df.rename(columns=metaedge_to_abbreviation)
df_disease_degree_sliced = disease_degree_df[disease_degree_df['disease_id'].isin(list(set(unique_diseases)))]
print(df_disease_degree_sliced.head())

compound_degree_df = compound_degree_df.rename(columns={'node_id': 'compound_id'}).drop('node_name', axis='columns')
compound_degree_df = compound_degree_df.rename(columns=metaedge_to_abbreviation)
df_compound_degree_sliced = compound_degree_df[compound_degree_df['compound_id'].isin(list(set(unique_compounds)))]
print(df_compound_degree_sliced.head())

    disease_id  DaG  DtC
12  DOID:10652  196    4
20  DOID:11119   16    0
37  DOID:12361   34    2
49  DOID:13378   14    1
55  DOID:14330  143    0
    compound_id  CbG  CcSE  CdG  CtD  CuG
46      DB00158   12    31    0    0    0
51      DB00163   19     0    6    0    1
77      DB00190    1   162   16    0    6
101     DB00215   14   823    8    1    2
106     DB00220   14   130    5    1   42


In [16]:
compound_degree_df.to_csv('compound_degree.csv', index=False, sep=',')
disease_degree_df.to_csv('disease_degree.csv', index=False, sep=',')

In [17]:
# Read degree features
compound_degree_df = pd.read_csv('compound_degree.csv')
disease_degree_df = pd.read_csv('disease_degree.csv')

for df in compound_degree_df, disease_degree_df:
    df.rename(columns={k: 'degree_' + k for k in df.columns[1:]}, inplace=True)

disease_degree_df.head(2)

,disease_id,degree_DaG,degree_DtC
0,DOID:0050156,18,0
1,DOID:0050425,12,0


In [18]:
feature_mat_df = pair_df.merge(compound_degree_df).merge(disease_degree_df).merge(dwpc_mat_df)
print(feature_mat_df.head())

  compound_id compound_name disease_id     disease_name category  status  \
0     DB00158    Folic Acid  DOID:8778  Crohn's disease      NOT       0   
1     DB00163     Vitamin E  DOID:8778  Crohn's disease      NaN       0   
2     DB00190     Carbidopa  DOID:8778  Crohn's disease      NaN       0   
3     DB00215    Citalopram  DOID:8778  Crohn's disease      NaN       0   
4     DB00220    Nelfinavir  DOID:8778  Crohn's disease      NaN       0   

   degree_CbG  degree_CcSE  degree_CdG  degree_CtD  degree_CuG  degree_DaG  \
0          12           31           0           0           0         120   
1          19            0           6           0           1         120   
2           1          162          16           0           6         120   
3          14          823           8           1           2         120   
4          14          130           5           1          42         120   

   degree_DtC  PC_CdrGiGaD  PC_CsCtD  PC_CtGaD  PC_CurGiGaD  
0           

In [19]:
with open('features.csv', 'wt') as write_file:
    feature_mat_df.to_csv(write_file, sep=',', index=False, float_format='%.4g')

In [20]:
### Transform Features

trans_df = feature_mat_df.copy()
degree_features = list(trans_df.columns[trans_df.columns.str.startswith('degree_')])
pc_features = list(trans_df.columns[trans_df.columns.str.startswith('PC_')])


# Standardize all features besides the prior
for feature in degree_features + pc_features:
    x = trans_df[feature]
    trans_df[feature] = (x - x.mean()) / x.std()

trans_df.head(3)

,compound_id,compound_name,disease_id,disease_name,category,status,degree_CbG,degree_CcSE,degree_CdG,degree_CtD,degree_CuG,degree_DaG,degree_DtC,PC_CdrGiGaD,PC_CsCtD,PC_CtGaD,PC_CurGiGaD
0,DB00158,Folic Acid,DOID:8778,Crohn's disease,NOT,0,0.085444,-0.957990,-0.626372,-0.559632,-0.525224,0.827135,0.505303,-0.208940,-0.561009,-0.404697,-0.2602
1,DB00163,Vitamin E,DOID:8778,Crohn's disease,NaN,0,0.932764,-1.125718,-0.008481,-0.559632,-0.436292,0.827135,0.505303,-0.208940,-0.670042,-0.404697,-0.2602
2,DB00190,Carbidopa,DOID:8778,Crohn's disease,NaN,0,-1.246059,-0.249205,1.021337,-0.559632,0.008370,0.827135,0.505303,0.290558,-0.259628,-0.404697,-0.2602


In [21]:
with open('transformed-features.csv', 'wt') as write_file:
    trans_df.to_csv(write_file, sep=',', index=False, float_format='%.4g')